In [1]:
import numpy as np
from scipy.io import loadmat
import src
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import scipy

In [2]:
def load_keypoints(file_path):
    data = loadmat(file_path)
    kp = data['kp']  #Keypoints (Nx2 matrix)
    desc = data['desc']  # Descritores(NxD matrix)
    return kp, desc

In [3]:
def match_keypoints_ransac(desc1, desc2, kp1, kp2, threshold=0.25, max_iter=1, inlier_threshold=2.0):
    
    # calcula a distncia entre os descritores
    distances = cdist(desc1, desc2)
    matches = []

    for i, row in enumerate(distances):
        sorted_indices = np.argsort(row)
        if row[sorted_indices[0]] < threshold * row[sorted_indices[1]]:
            matches.append((i, sorted_indices[0]))

    matches = np.array(matches)

    
    best_inliers = []
    best_H = None
    
    print("matches: ", matches.shape)

    for _ in range(max_iter):
        print("hello")
        #4 matches aleatórios (usar mais?)
        sampled_matches = matches[np.random.choice(matches.shape[0], 8, replace=False)]
        src_pts = kp1[sampled_matches[:, 0]]
        dst_pts = kp2[sampled_matches[:, 1]]

        H = src.getPerspectiveTransform(src_pts,dst_pts)
        
        inliers = []
        for match in matches:
            pt1 = kp1[match[0]]
            pt2 = kp2[match[1]]
            
            
            pt1_hom = np.append(pt1, 1)  
            transformed_pt = H @ pt1_hom
            transformed_pt /= transformed_pt[2]  
            
            
            dist = np.linalg.norm(transformed_pt[:2] - pt2)
            if dist < inlier_threshold:
                inliers.append(match)

        print("inliers: ", inliers)
        
        # Se o número de inliers for maior que o melhor encontrado, atualizar
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_H = H

    return best_inliers, best_H, matches

In [4]:
class DMatch:
    def __init__(self, _queryIdx, _trainIdx, _distance):
        self.queryIdx = _queryIdx
        self.trainIdx = _trainIdx
        self.distance = _distance
    
    
    def __repr__(self):
        return 'DMatch(queryIdx: %s, trainIdx: %s, distance: %s)' % (self.queryIdx, self.trainIdx, self.distance)



def knn_match_vectorized(descriptors_query, descriptors_train, k):
    distances = scipy.spatial.distance.cdist(descriptors_query, descriptors_train, 'euclidean') ## calculate the distances between each query descriptor and each train descriptor: i -> query, j -> train
    indices = np.argsort(distances, axis=1) ## order for each row (query descriptor) the indices of the train descriptors
    matches = np.zeros((descriptors_query.shape[0], k), dtype=object)
    
    for i in range(descriptors_query.shape[0]):
        k_nearest_neighbors = [(index, distances[i, index]) for index in indices[i, :k]]
        match = np.array([DMatch(_queryIdx=i, _trainIdx=neighbor[0], _distance=neighbor[1]) for neighbor in k_nearest_neighbors])
        matches[i, :] = match
        
    return matches



def matching_optional(desc1, desc2, threshold=0.25):
    
    distances = scipy.spatial.distance.cdist(desc1, desc2, 'euclidean') ## calculate the distances between each query descriptor and each train descriptor: i -> query, j -> train
    sorted_indices = np.argsort(distances, axis=1)[:, :2]
    
    first_min_indices = sorted_indices[:, 0]
    second_min_indices = sorted_indices[:, 1]

    first_min_distances = distances[np.arange(distances.shape[0]), first_min_indices]
    second_min_distances = distances[np.arange(distances.shape[0]), second_min_indices]

    condition = first_min_distances < threshold * second_min_distances
    matches = np.column_stack((np.where(condition)[0], first_min_indices[condition]))
    
    return matches






In [5]:
# Inicializar o mosaico
initial_image_path = 'ISRwall/input_1/images/img_0001.jpg'
initial_image = src.image_to_matrix(initial_image_path)
width, height = (20000,10000)
start=2000
dst = np.full((height, width, initial_image.shape[2] if initial_image.ndim == 3 else 1), 0, dtype=initial_image.dtype)

for y in range(initial_image.shape[0]):
    for x in range(initial_image.shape[1]):
        if 0 <= x+12000 < width and 0 <= y+6000 < height:
            dst[y+6000, x+12000] = initial_image[y, x]

src.matrix_to_image(dst, f"final_mosaic.jpg")

In [6]:
counter = 0

kp1, desc1 = load_keypoints(f"ISRwall/input_1/keypoints/kp_000{counter+1}.mat")
kp2, desc2 = load_keypoints(f"ISRwall/input_1/keypoints/kp_000{counter+2}.mat")



print("kp1: ", kp1.shape)
print("desc1: ", desc1.shape)
print("kp2: ", kp2.shape)
print("desc2: ", desc2.shape)

kp1:  (6106, 2)
desc1:  (6106, 64)
kp2:  (6808, 2)
desc2:  (6808, 64)


In [7]:

#matches = match_keypoints(desc1, desc2)

inliers_original, H_ransac_original, matches_original = match_keypoints_ransac(desc1, desc2, kp1, kp2)

matches:  (170, 2)
hello
inliers:  [array([32,  4]), array([154,  49]), array([279, 134]), array([364, 175]), array([435, 217]), array([443, 121]), array([527, 417]), array([542, 228]), array([807, 430]), array([824, 319]), array([953, 347]), array([986, 421]), array([1050,  512]), array([1070,  488]), array([1081,  443]), array([1152,  824]), array([1273,  603]), array([1319,  687]), array([1433, 1183]), array([1450,  936]), array([1477,  843]), array([1497, 1030]), array([1502,  854]), array([1512,  745]), array([1530,  873]), array([1531,  771]), array([1537,  707]), array([1539, 1043]), array([1630, 1181]), array([1734, 1017]), array([1786, 1274]), array([1842,  840]), array([1963, 1397]), array([2060, 1422]), array([2062, 1265]), array([2074, 1561]), array([2132, 1738]), array([2300, 1799]), array([2397, 1548]), array([2406, 1675]), array([2429, 1656]), array([2528, 2054]), array([2569, 1812]), array([2598, 2376]), array([2638, 2171]), array([2686, 2393]), array([2730, 2049]), arr

In [8]:
print("matches_original: ", matches_original.shape)
print("inliers: ", len(inliers_original))
print("H_ransac: ", H_ransac_original)

matches_original:  (170, 2)
inliers:  85
H_ransac:  [[ 8.06496679e-01 -1.65600742e-01  7.31523669e+02]
 [-3.23721837e-02  7.73650869e-01  9.26574433e+02]
 [-2.94897264e-05 -6.40383448e-05  1.00000000e+00]]


In [9]:
matches_2 = matching_optional(desc1, desc2)

In [10]:
## choice 1
inlier_threshold = 2.0
sampled_matches = matches_original[np.random.choice(matches_original.shape[0], 4, replace=False)]
src_pts = kp1[sampled_matches[:, 0]]
dst_pts = kp2[sampled_matches[:, 1]]

H = src.getPerspectiveTransform(src_pts,dst_pts)
        
inliers = []
for match in matches_original:
    pt1 = kp1[match[0]]
    pt2 = kp2[match[1]]
            
            
    pt1_hom = np.append(pt1, 1)  
    transformed_pt = H @ pt1_hom
    transformed_pt /= transformed_pt[2]  

    dist = np.linalg.norm(transformed_pt[:2] - pt2)
    if dist < inlier_threshold:
        inliers.append(match)

inliers_original = np.array(inliers)

print("inliers: ", inliers_original.shape)
print(inliers_original[-10:])
        

inliers:  (18, 2)
[[2878 2483]
 [3143 2804]
 [3198 2806]
 [3505 2983]
 [3724 3553]
 [4062 3758]
 [4181 4894]
 [4521 4365]
 [4857 4865]
 [5073 4968]]


In [11]:
### vectorized version of ransac

inlier_threshold = 2.0
best_inliers = []
best_H = None

# samples_matches = matches_2[np.random.choice(matches_2.shape[0], 4, replace=False)]

H = src.getPerspectiveTransform(kp1[sampled_matches[:, 0]], kp2[sampled_matches[:, 1]])

src_pts_hom = np.column_stack((kp1[matches_original[:, 0]], np.ones((matches_original.shape[0], 1)))).T

print(src_pts_hom[:, :10])

# print("src_pts_hom: ", src_pts_hom.shape)
# print("H: ", H.shape)

transformed_pts_hom = (H @ src_pts_hom).T


transformed_pts_hom /= transformed_pts_hom[:, 2][:, None]

dst_pts_all = kp2[matches_original[:, 1]]

print(transformed_pts_hom[:, :2].shape)
print(dst_pts_all.shape)

dists = np.linalg.norm(transformed_pts_hom[:, :2] - dst_pts_all, axis=1)

inliers_2 = matches_original[dists < inlier_threshold]

print("inliers: ", inliers_2.shape)
print(inliers_2[-10:])


[[1.52867529e+03 2.80704980e+03 1.51490808e+03 2.11259399e+03
  2.19201953e+03 2.65929443e+03 1.98127258e+03 1.77079089e+03
  2.13739819e+03 2.15568701e+03]
 [1.10878992e+03 1.06965356e+03 8.26793152e+02 8.11127136e+02
  8.09396790e+02 1.12781799e+03 8.35782349e+02 1.28695178e+03
  1.35662537e+03 1.31542883e+03]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00]]
(170, 2)
(170, 2)
inliers:  (18, 2)
[[2878 2483]
 [3143 2804]
 [3198 2806]
 [3505 2983]
 [3724 3553]
 [4062 3758]
 [4181 4894]
 [4521 4365]
 [4857 4865]
 [5073 4968]]
